### Step 1: Data Collection Using Spotify API

In [57]:
# Imports woooo!
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import collections
import pandas as pd

In [2]:
# Scopes to define Spotify permissions
scope = 'user-library-read playlist-read-private user-top-read'

In [4]:
# Initiate API Query Object
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,client_secret=client_secret, redirect_uri=redirect_uri,scope=scope))

In [5]:
# Initial check if API is working as intended
sp.current_user()

{'display_name': 'ericwyeah',
 'external_urls': {'spotify': 'https://open.spotify.com/user/ericwyeah'},
 'followers': {'href': None, 'total': 15},
 'href': 'https://api.spotify.com/v1/users/ericwyeah',
 'id': 'ericwyeah',
 'images': [{'height': None,
   'url': 'https://i.scdn.co/image/ab6775700000ee850608af8b68d924da27603720',
   'width': None}],
 'type': 'user',
 'uri': 'spotify:user:ericwyeah'}

### Step 2: Pull Current User's Top Tracks

In [6]:
top_query = sp.current_user_top_tracks(limit=20, offset=0, time_range='medium_term')['items']

In [7]:
# Create function to retrieve list of top tracks for current user
def get_top_tracks():
    top_tracks = []
    
    base_uri = 'https://api.spotify.com/v1'

    top_query = sp.current_user_top_tracks(limit=20, offset=0, time_range='medium_term')['items']
    for entry in top_query:
        artists = []
        for artist in entry['artists']:
            artists.append(artist['name'])
        top_tracks.append({'artists': artists, 'track_id': entry['id'], 'name': entry['name'], 'preview_url': entry['preview_url'], 'image': entry['album']['images'][-1]['url']})
    
    for track in top_tracks:
        if len(track['artists']) > 1:
            track['display_info'] = track['name'] + ' by ' + ' and '.join(track['artists'])
        else:
            track['display_info'] = track['name'] + ' by ' + track['artists'][0]
    
    return top_tracks

In [8]:
top_tracks = get_top_tracks()
top_tracks[0]

{'artists': ['Magic Man'],
 'track_id': '4EaO2XR7gWaHq8DW7nu1iY',
 'name': 'Paris',
 'preview_url': 'https://p.scdn.co/mp3-preview/5d6ff173f44b7efea949d27adf931fc541f384c9?cid=d3f619ebc4e148348b7c292d61418501',
 'image': 'https://i.scdn.co/image/ab67616d00004851c0d5f377e45b39d3ea7c7b67',
 'display_info': 'Paris by Magic Man'}

In [9]:
# Create function to format tracks to include track name and artist name
def format_tracks(track_list):
    display_text = []

    for track in track_list:
        if len(track['artists']) > 1:
            display_text.append(track['name'] + ' by ' + ' and '.join(track['artists']))
        else:
            display_text.append(track['name'] + ' by ' + track['artists'][0])
    return display_text

In [10]:
format_tracks(top_tracks)[0]

'Paris by Magic Man'

### Step 3: Test Search Functionality (for implementation in web app)

In [11]:
# Example query of Lizzo
query = 'Lizzo'

In [12]:
# Create function to return list of search results based on search query
def search_tracks(query):
    search_results = sp.search(q=query, type='track,artist')
    search_tracks = []

    for entry in search_results['tracks']['items']:
        artists = []
        for artist in entry['artists']:
            artists.append(artist['name'])
        search_tracks.append({'artists': artists, 'track_id': entry['id'], 'name': entry['name'], 'preview_url': entry['preview_url'], 'image': entry['album']['images'][-1]['url']})
    return search_tracks

In [13]:
search_results = search_tracks(query)
search_results[0]

{'artists': ['Lizzo', 'Cardi B'],
 'track_id': '6KgtcmCF9Ky68XC7ezxl3s',
 'name': 'Rumors (feat. Cardi B)',
 'preview_url': 'https://p.scdn.co/mp3-preview/252a7be97bc9172a9aca8552abf964783bc24cec?cid=d3f619ebc4e148348b7c292d61418501',
 'image': 'https://i.scdn.co/image/ab67616d00004851324290e5b99a797b99e247bb'}

In [14]:
format_tracks(search_results)[0]

'Rumors (feat. Cardi B) by Lizzo and Cardi B'

### Step 4: Test Retrieving Current User's Playlists and their Tracks (for implementation in web app)

In [15]:
# Use Spotify API Object to pull current user's playlists
playlists = sp.current_user_playlists(limit=50, offset=0)['items']

In [16]:
# Format playlist information for user viewing
playlist_display = [entry['name'] + ' by ' + entry['owner']['display_name'] for entry in playlists]

In [17]:
playlist_display[0]

'My Playlist by ericwyeah'

In [18]:
# Create list of current user's playlists
def query_playlists():
    playlists_list = []
    playlists = sp.current_user_playlists(limit=50, offset=0)['items']
    for playlist in playlists:
        playlists_list.append({'name': playlist['name'], 'display_info': (playlist['name'] + ' by ' + playlist['owner']['display_name']), 'owner': playlist['owner']['display_name'], 'playlist_id': playlist['id'], 'image': playlist['images'][-1]['url']})
    return playlists_list

# Format user's playlists
def format_playlists(playlists_list):
    formatted_playlists = [entry['name'] + ' by ' + entry['owner'] for entry in playlists_list]
    return formatted_playlists

In [19]:
playlist_list = query_playlists()
format_playlists(playlist_list)[0]

'My Playlist by ericwyeah'

In [20]:
playlist_list[0]

{'name': 'My Playlist',
 'display_info': 'My Playlist by ericwyeah',
 'owner': 'ericwyeah',
 'playlist_id': '75fV6pcpgTv0xLOadc343I',
 'image': 'https://i.scdn.co/image/ab67616d0000b2733a2cc6c5cdd9f07a95efa51b'}

In [21]:
# Create function that returns list of tracks in specified playlist
def tracks_in_playlist(username, playlist_id):
    playlist_tracks = []

    playlist_query = sp.user_playlist_tracks(username, playlist_id)['items']
    for entry in playlist_query:
        artists = []
        for artist in entry['track']['artists']:
            artists.append(artist['name'])

        playlist_tracks.append({'artists': artists, 'track_id': entry['track']['id'], 'name': entry['track']['name'], 'preview_url': entry['track']['preview_url'], 'image': entry['track']['album']['images'][-1]['url']})
    return playlist_tracks

In [70]:
sp.search(q='My Playlist',type='playlist')['playlists']['items'][0]

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/75fV6pcpgTv0xLOadc343I'},
 'href': 'https://api.spotify.com/v1/playlists/75fV6pcpgTv0xLOadc343I',
 'id': '75fV6pcpgTv0xLOadc343I',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b2733a2cc6c5cdd9f07a95efa51b',
   'width': 640}],
 'name': 'My Playlist',
 'owner': {'display_name': 'ericwyeah',
  'external_urls': {'spotify': 'https://open.spotify.com/user/ericwyeah'},
  'href': 'https://api.spotify.com/v1/users/ericwyeah',
  'id': 'ericwyeah',
  'type': 'user',
  'uri': 'spotify:user:ericwyeah'},
 'primary_color': None,
 'public': None,
 'snapshot_id': 'MywzMWQ4MDkxOGE2YjAxNDkwMjExZDFjNWNiMmVhZjM5YTEwMDc0Y2Yx',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/75fV6pcpgTv0xLOadc343I/tracks',
  'total': 1},
 'type': 'playlist',
 'uri': 'spotify:playlist:75fV6pcpgTv0xLOadc343I'}

In [23]:
def query_playlist_id(playlist_name, playlist_list):
    return next(item for item in playlist_list if item["name"] == playlist_name)['playlist_id']

In [24]:
query_playlist_id('My Playlist', playlist_list)

'75fV6pcpgTv0xLOadc343I'

In [25]:
# Name of first track in playlist ;)
tracks_in_playlist('ericwyeah', playlist_list[0]['playlist_id'])[0]['name']

'Hello World'

### Step 5: Create a Function to Evaluate Tracks' Features

In [26]:
def get_track_features(track_ids):
    audio_features = sp.audio_features(track_ids)
    return audio_features

In [27]:
top_tracks_ids = [entry['track_id'] for entry in top_tracks]
top_track_features = get_track_features(top_tracks_ids)

In [28]:
def average_features(list_of_features):
    numeric_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence', 'tempo']
    numeric_values = [{ your_key: dict_entry[your_key] for your_key in numeric_features } for dict_entry in list_of_features]
    
    counter = collections.Counter()
    for d in numeric_values: 
        counter.update(d)
        
    return {k: v / len(list_of_features) for k, v in dict(counter).items()}

In [29]:
average_features(top_track_features)

{'acousticness': 0.23316649999999997,
 'danceability': 0.55505,
 'energy': 0.6758,
 'instrumentalness': 0.009247315,
 'liveness': 0.138085,
 'loudness': -6.33005,
 'speechiness': 0.07387,
 'valence': 0.4647999999999999,
 'tempo': 114.1386}

### Step 6: Identify the Average Features of Different Spotify Genres

Spotify's Today's Top Hits Playlist is a collection of the current top Pop hits. In order to determine the average feature values of the Pop genre, we used this playlist as reference.

In [30]:
def playlist_average_features(playlist_id):
    tracks = tracks_in_playlist('ericwyeah',playlist_id)
    track_ids = [entry['track_id'] for entry in tracks]
    tracks_features = get_track_features(track_ids)
    return average_features(tracks_features)

In [31]:
genres = {}

#### POP

In [32]:
todays_top_hits_id = '37i9dQZF1DXcBWIGoYBM5M'

In [33]:
playlist_average_features(todays_top_hits_id)

{'acousticness': 0.22150759999999994,
 'danceability': 0.6958200000000001,
 'energy': 0.63538,
 'instrumentalness': 0.0013480440000000003,
 'liveness': 0.180602,
 'loudness': -6.365960000000001,
 'speechiness': 0.09286200000000003,
 'valence': 0.5238920000000002,
 'tempo': 120.58756000000002}

In [34]:
genres['pop'] = playlist_average_features(todays_top_hits_id)

##### Classical

In [35]:
classical_essentials_id = '37i9dQZF1DWWEJlAGA9gs0'

In [36]:
playlist_average_features(classical_essentials_id)

{'acousticness': 0.9239100000000002,
 'danceability': 0.2489889999999999,
 'energy': 0.09592900000000001,
 'instrumentalness': 0.7452943240000001,
 'liveness': 0.12242199999999999,
 'loudness': -23.397980000000004,
 'speechiness': 0.04372200000000001,
 'valence': 0.12267999999999998,
 'tempo': 99.69890000000001}

In [37]:
genres['classical'] = playlist_average_features(classical_essentials_id)

##### EDM

In [38]:
edm_hits_id = '37i9dQZF1DX3Kdv0IChEm9'

In [39]:
playlist_average_features(edm_hits_id)

{'acousticness': 0.11803159999999997,
 'danceability': 0.6192000000000001,
 'energy': 0.7714833333333335,
 'instrumentalness': 0.026072740833333333,
 'liveness': 0.21089666666666668,
 'loudness': -5.099099999999999,
 'speechiness': 0.05259500000000002,
 'valence': 0.4325233333333333,
 'tempo': 121.16761666666665}

In [40]:
genres['edm'] = playlist_average_features(edm_hits_id)

##### Indie

In [41]:
indie_id = '37i9dQZF1DX26DKvjp0s9M'

In [42]:
playlist_average_features(indie_id)

{'acousticness': 0.245093796,
 'danceability': 0.5536,
 'energy': 0.6521400000000003,
 'instrumentalness': 0.08273723770000001,
 'liveness': 0.17088599999999998,
 'loudness': -7.147670000000002,
 'speechiness': 0.045124999999999985,
 'valence': 0.468014,
 'tempo': 120.31647}

In [43]:
genres['indie'] = playlist_average_features(indie_id)

##### Soft Pop

In [44]:
soft_pop_id = '37i9dQZF1DWTwnEm1IYyoj'

In [45]:
playlist_average_features(soft_pop_id)

{'acousticness': 0.33000749999999995,
 'danceability': 0.5815800000000002,
 'energy': 0.5568550000000002,
 'instrumentalness': 0.0005572250000000001,
 'liveness': 0.13098099999999996,
 'loudness': -6.761510000000002,
 'speechiness': 0.052465000000000005,
 'valence': 0.40887000000000007,
 'tempo': 114.90280999999997}

In [46]:
genres['soft_pop'] = playlist_average_features(soft_pop_id)

##### R & B

In [47]:
r_and_b_id = '37i9dQZF1DX4SBhb3fqCJd'

In [48]:
playlist_average_features(r_and_b_id)

{'acousticness': 0.27255300000000005,
 'danceability': 0.6129999999999999,
 'energy': 0.5653600000000001,
 'instrumentalness': 0.001224497,
 'liveness': 0.185966,
 'loudness': -7.36042,
 'speechiness': 0.10812399999999998,
 'valence': 0.4450159999999999,
 'tempo': 111.85530000000003}

In [49]:
genres['rnb'] = playlist_average_features(r_and_b_id)

##### Hip Hop

In [50]:
hip_hop_id = '37i9dQZF1DXbkfWVLd8wE3'

In [51]:
playlist_average_features(hip_hop_id)

{'acousticness': 0.12341976041666665,
 'danceability': 0.7577083333333331,
 'energy': 0.7451041666666667,
 'instrumentalness': 0.003262283854166666,
 'liveness': 0.20602291666666675,
 'loudness': -5.638437500000002,
 'speechiness': 0.22962291666666682,
 'valence': 0.6421145833333334,
 'tempo': 109.14435416666662}

In [52]:
genres['hip_hop'] = playlist_average_features(hip_hop_id)

##### Feelin Myself

In [53]:
feelin_myself_id = '37i9dQZF1DX6GwdWRQMQpq'

In [54]:
playlist_average_features(feelin_myself_id)

{'acousticness': 0.12620846000000002,
 'danceability': 0.8277400000000001,
 'energy': 0.6111599999999999,
 'instrumentalness': 0.0144701232,
 'liveness': 0.19104399999999994,
 'loudness': -6.675579999999999,
 'speechiness': 0.22947800000000007,
 'valence': 0.5526599999999999,
 'tempo': 118.62462000000001}

In [55]:
genres['feelin_myself'] = playlist_average_features(feelin_myself_id)

### Step 7: Putting Everything Together <3

In [58]:
df = pd.DataFrame(genres)

In [59]:
transposed = df.T

In [60]:
df

,pop,classical,edm,indie,soft_pop,rnb,hip_hop,feelin_myself
acousticness,0.221508,0.923910,0.118032,0.245094,0.330007,0.272553,0.123420,0.126208
danceability,0.695820,0.248989,0.619200,0.553600,0.581580,0.613000,0.757708,0.827740
energy,0.635380,0.095929,0.771483,0.652140,0.556855,0.565360,0.745104,0.611160
instrumentalness,0.001348,0.745294,0.026073,0.082737,0.000557,0.001224,0.003262,0.014470
liveness,0.180602,0.122422,0.210897,0.170886,0.130981,0.185966,0.206023,0.191044
loudness,-6.365960,-23.397980,-5.099100,-7.147670,-6.761510,-7.360420,-5.638438,-6.675580
speechiness,0.092862,0.043722,0.052595,0.045125,0.052465,0.108124,0.229623,0.229478
valence,0.523892,0.122680,0.432523,0.468014,0.408870,0.445016,0.642115,0.552660
tempo,120.587560,99.698900,121.167617,120.316470,114.902810,111.855300,109.144354,118.624620


In [61]:
# Formatted Search Results
format_tracks(search_results)

['Rumors (feat. Cardi B) by Lizzo and Cardi B',
 'Good as Hell (feat. Ariana Grande) by Lizzo and Ariana Grande',
 'Truth Hurts by Lizzo',
 'Boys by Lizzo',
 'Juice by Lizzo',
 'Good as Hell by Lizzo',
 'Good as Hell by Lizzo',
 'Good as Hell (feat. Ariana Grande) - Remix by Lizzo and Ariana Grande',
 'Boys by Lizzo',
 'Tempo (feat. Missy Elliott) by Lizzo and Missy Elliott']

In [62]:
# Formatted Top Tracks
format_tracks(top_tracks)

['Paris by Magic Man',
 'Praying by Kesha',
 'Fantasy by Alina Baraz and Galimatias',
 'Shed a Light by Robin Schulz and David Guetta and Cheat Codes',
 'T-Shirt Weather by Circa Waves',
 'Between The Raindrops by Lifehouse and Natasha Bedingfield',
 'Beneath Your Beautiful (feat. Emeli Sandé) by Labrinth and Emeli Sandé',
 'Marry Me by Train',
 'Peaches (feat. Daniel Caesar & Giveon) by Justin Bieber and Daniel Caesar and Giveon',
 'Sound Of Your Heart by Shawn Hook',
 'Kung Foo Fighting by October Cherries',
 'Roots by Imagine Dragons',
 'On Hold by The xx',
 'Temporary Bliss by The Cab',
 'Little Wonders by Rob Thomas',
 "You're Beautiful by James Blunt",
 'It Ain’t Me (with Selena Gomez) by Kygo and Selena Gomez',
 'Sad Song (feat. Elena Coats) by We The Kings and Elena Coats',
 'Shape of You by Ed Sheeran',
 'Undercover Martyn by Two Door Cinema Club']

In [63]:
# Formatted Playlist List
format_playlists(playlist_list)

['My Playlist by ericwyeah',
 'my kpop roots by Stephanie Kwan',
 'permanent smile by mariyahh54',
 'Ghibli Music Box by Melanie メラニー',
 'used to be friends, strangers now by Aleena Shad',
 'the hulk inside by mariyahh54',
 'Chill Vibes by Spotify',
 'Peaceful Piano by Spotify',
 'Dreamy Vibes by Spotify',
 'welcome to my mind by Aleena Shad',
 'eventually by mariyahh54',
 'barcelona 2016 by tina',
 "if i can hear you, it's not loud enough by Aleena Shad",
 "cupid should've used a bullet by Aleena Shad",
 'hey bb by ericwyeah',
 'Piano by ericwyeah',
 '  by Melissa Viernes',
 'Guardians of the Galaxy - Star-Lord: Awesome Mix Vol. 1 (Plus songs cut from the film) by Dan Zetterstrom',
 'Liked from Radio by ericwyeah',
 'Mood Booster by Spotify',
 'Totally Stress Free by Spotify']

In [64]:
tracks_in_playlist(sp.current_user()['id'], query_playlist_id('eventually', playlist_list))

[{'artists': ['Kygo', 'Will Heard'],
  'track_id': '0j0DNujXWeupLpZobbABoo',
  'name': 'Nothing Left (feat. Will Heard)',
  'preview_url': 'https://p.scdn.co/mp3-preview/b8ab3ba12d30be95dacc171d7c4cde03405015d2?cid=d3f619ebc4e148348b7c292d61418501',
  'image': 'https://i.scdn.co/image/ab67616d00004851ff34611201eb1d4eed0cddbd'},
 {'artists': ['Emmit Fenn'],
  'track_id': '6kUohQ7py1XEcQAhUgf2Zz',
  'name': 'Painting Greys',
  'preview_url': None,
  'image': 'https://i.scdn.co/image/ab67616d00004851003dca9f70f88e8940fba146'},
 {'artists': ['Against The Current', 'Taka'],
  'track_id': '3lbtBnZlPTSlcQjgvUGsmu',
  'name': 'Dreaming Alone',
  'preview_url': 'https://p.scdn.co/mp3-preview/fa58b85583d416fdb333fc854c11c2bd1d4f49df?cid=d3f619ebc4e148348b7c292d61418501',
  'image': 'https://i.scdn.co/image/ab67616d00004851cb7523307aac3e10b66bef40'},
 {'artists': ['FRND'],
  'track_id': '7qDQhmvXb27hqKQxZ49GYr',
  'name': 'Friend',
  'preview_url': 'https://p.scdn.co/mp3-preview/18ea4400d74d0f694